In [ ]:
!pip install -qU \
  langchain-core \
  langchain-google-genai \
  langchain-community \
  langgraph

%load_ext autoreload
%autoreload 2

In [ ]:
from config import set_environment

set_environment("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

In [ ]:
job_description : str = """
A Java Developer designs, develops, tests, and maintains Java-based applications, ensuring high performance, scalability, and security. Responsibilities include writing clean code, collaborating with cross-functional teams, troubleshooting issues, integrating with databases and APIs, and documenting technical processes. Key requirements are strong Java and framework (e.g., Spring, Hibernate) proficiency, database skills, and experience with version control (e.g., Git). 
Key Responsibilities
Design and Development: Create and implement robust, high-volume, and low-latency Java applications. 
Code Quality: Write clean, efficient, testable, and scalable code following best practices. 
Requirement Analysis: Analyze user requirements and define system objectives and functionality. 
Integration: Integrate applications with databases and third-party services using APIs. 
Testing & Debugging: Conduct thorough testing, debug complex issues, and resolve application problems. 
Deployment: Participate in the full software development lifecycle, including deployment and maintenance. 
Documentation: Prepare detailed documentation for application designs, technical specifications, and user support. 
Team Collaboration: Work with cross-functional teams, such as designers, product owners, and other engineers, to deliver applications. 

"""

prompt_template = (
    """
    Given a job description, decide whether it suits a junior Java
    developer.
    Job Description:

    {job_description}
    """
)

result = llm.invoke(prompt_template.format(job_description = job_description))

print(result.content)



In [ ]:
prompt_template_enum = (
    """
    Given a job description, decide whether it suits a junior Java
    developer.
    Job Description:

    {job_description}

    Answer only YES or NO
    """
)

result = llm.invoke(prompt_template_enum.format(job_description = job_description))

print(result.content)

In [ ]:
from enum import Enum

from langchain.output_parsers import EnumOutputParser
from langchain_core.messages import HumanMessage

In [ ]:
class IsSuitableJobEnum(Enum):
    YES = "YES"
    NO = "NO"

parser = EnumOutputParser(enum=IsSuitableJobEnum)

assert parser.invoke("NO") == IsSuitableJobEnum.NO
assert parser.invoke("YES\n") == IsSuitableJobEnum.YES
assert parser.invoke("YES \n") == IsSuitableJobEnum.YES


In [ ]:
chain = llm | parser

result = chain.invoke(prompt_template_enum.format(job_description = job_description))

print(result)

In [ ]:
from typing_extensions import TypedDict
from langchain_core.runnables.config import RunnableConfig
from langgraph.graph import StateGraph, START, END

class JobApplicationState(TypedDict):
    job_description: str
    is_suitable: IsSuitableJobEnum
    application: str


analyze_chain = llm | parser

def analyze_job_description(state):
    prompt = prompt_template_enum.format(job_description = state["job_description"])
    result = analyze_chain.invoke(prompt)
    return {
        "is_suitable": result
    }


 

def is_suitable_condition(state):
    return state["is_suitable"] == IsSuitableJobEnum.YES

def generate_application(state):
    print("...generating application...")
    return {"application": "some_fake_application", "actions": ["action2"]}
    
builder = StateGraph(JobApplicationState)

builder.add_node("analyze_job_description", analyze_job_description)
builder.add_node("generate_application", generate_application)

builder.add_edge(START, "analyze_job_description")
builder.add_conditional_edges(
    "analyze_job_description", is_suitable_condition,
    {
        True: "generate_application",
        False: END
    }
)

builder.add_edge("generate_application", END)
graph = builder.compile()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))



In [ ]:
graph.invoke({
    "job_description": job_description
})

In [ ]:
from typing_extensions import TypedDict
from langchain_core.runnables.config import RunnableConfig
from langgraph.graph import StateGraph, START, END
import logging


logger = logging.getLogger(__name__)

class JobApplicationState(TypedDict):
    job_description: str
    is_suitable: IsSuitableJobEnum
    application: str


analyze_chain = llm | parser

def analyze_job_description(state):
    try:
        prompt = prompt_template_enum.format(job_description = state["job_description"])
        result = analyze_chain.invoke(prompt)
        return {
            "is_suitable": result
        }
    except Exception as e:
        logger.error(f"Exception {e} occured while executing analyze_job_description")
        return {
            "is_suitable": IsSuitableJobEnum.NO
        }


 

def is_suitable_condition(state):
    return state["is_suitable"] == IsSuitableJobEnum.YES

def generate_application(state):
    print("...generating application...")
    return {"application": "some_fake_application", "actions": ["action2"]}
    
builder = StateGraph(JobApplicationState)

builder.add_node("analyze_job_description", analyze_job_description)
builder.add_node("generate_application", generate_application)

builder.add_edge(START, "analyze_job_description")
builder.add_conditional_edges(
    "analyze_job_description", is_suitable_condition,
    {
        True: "generate_application",
        False: END
    }
)

builder.add_edge("generate_application", END)
graph = builder.compile()

from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))



In [ ]:
graph.invoke({
    "job_description": "Not a job application"
})